In [1]:
import Pkg; 
Pkg.activate(@__DIR__); 
Pkg.instantiate()

  Activating project at `~/art/spline-trajectory-optimization/julia`


In [2]:
include("utils/SplineTrajectory.jl")
include("utils/DiscreteTrajectory.jl")
include("utils/IO.jl")

load_raw_trajectory (generic function with 1 method)

In [3]:
import ForwardDiff as FD
import BSplineKit as BK
import PyPlot as plt
plt.pygui(true)
using Test
using DelimitedFiles
using OSQP
using SparseArrays
using Statistics
Drawing()

In [4]:
# ENV["PYTHON"]="/home/haoru/art/spline_traj_opt_virtualenv/bin/python3"; Pkg.build("PyCall");

In [5]:
function eval_coeffs(
    basis::BK.PeriodicBSplineBasis,
    coeffs::AbstractVector{T},
    ts::AbstractVector{Float64},
    op = BK.Derivative(0)) where {T}
    N = length(ts)
    x = zeros(T, N)
    k = typeof(basis).parameters[1]
    for i=1:N
        ti = ts[i]
        idx, bs = basis(ti, op)
        for j=1:k
            x[i] += bs[j] * coeffs[idx-j+1]
        end
    end
    return x
end
    

eval_coeffs (generic function with 2 methods)

In [6]:
@testset "Curve reconstruction" begin
# Let's verify that the control points have a linear effect on the final curve shape
# i.e. we can reconstruct the curve by multiplying the jacobians with the coefficients

x, y, z = load_raw_trajectory("examples/race_track/monza/MONZA_UNOPTIMIZED_LINE_enu.csv")
traj = SplineTrajectory(x, y, 5)
ts = [range(0.0, step = 0.01, stop = 0.99);]
coeffs_x = copy(traj.spl_x.coefs)
coeffs_y = copy(traj.spl_y.coefs)
basis = traj.spl_x.basis
dx = FD.jacobian(coef->eval_coeffs(basis, coef, ts), coeffs_x)
dy = FD.jacobian(coef->eval_coeffs(basis, coef, ts), coeffs_y)

x_recon = dx * coeffs_x.data
y_recon = dy * coeffs_y.data

x_intp, y_intp = traj_ev(traj, ts)

@test dx == dy
@test norm(x_recon-x_intp) < 1e-9
@test norm(y_recon-y_intp) < 1e-9

end

Test Summary:        | Pass  Total  Time
Curve reconstruction |    3      3  6.0s


Test.DefaultTestSet("Curve reconstruction", Any[], 3, false, false, true, 1.694917368189596e9, 1.694917374142454e9, false)

Now let's define the minimum-curvature cost function.

In [7]:
function min_curvature_cost(
    Z::AbstractVector{T},
    traj_s::SplineTrajectory,
    ts::AbstractVector{Float64}) where {T}

    N = length(ts)
    
    Tx, Ty = traj_ev(traj_s, ts)
    n = length(Tx)
    distances = Float64[]
    for i in 1:n
        # Use modulo operator to connect the last point to the first point
        j = i % n + 1
        dx = Tx[j] - Tx[i]
        dy = Ty[j] - Ty[i]
        push!(distances, sqrt(dx^2 + dy^2))
    end

    dTx, dTy = traj_ev(traj_s, ts, BK.Derivative(1))
    d2Tx, d2Ty = traj_ev(traj_s, ts, BK.Derivative(2))

    # plot curvatures
    curvatures = (dTx  .* d2Ty .- dTy .* d2Tx) ./ (dTx .^ 2 + dTy .^ 2) .^ 1.5
    # @show size(curvatures)
    plt.plot(curvatures)

    v = ones(eltype(Z), N)
    denom = (dTx .^ 2 + dTy .^ 2) .^ 3
    demom = ones(eltype(Z), length(denom))
    Pxx = (dTy .^ 2 .* v .* distances) ./ denom
    Pxy = (-2.0 .* dTx .* dTy .* v .* distances) ./ denom
    Pyy = (dTx .^ 2 .* v .* distances) ./ denom

    Pxx = diagm(Pxx)
    Pxy = diagm(Pxy)
    Pyy = diagm(Pyy)

    B = FD.jacobian(coef->eval_coeffs(traj_s.spl_x.basis, coef, ts), traj_s.spl_x.coefs)
    # Bx = hcat(B, zeros(eltype(B), N, length(Z) ÷ 2))
    # By = hcat(zeros(eltype(B), N, length(Z) ÷ 2), B)

    # A = repeat(d2Tx, 1, length(traj_s.spl_x.coefs))
    d2B = FD.jacobian(coef->eval_coeffs(traj_s.spl_x.basis, coef, ts, BK.Derivative(2)), traj_s.spl_x.coefs)
    Bx = hcat(d2B, zeros(eltype(d2B), N, length(Z) ÷ 2))
    By = hcat(zeros(eltype(d2B), N, length(Z) ÷ 2), d2B)
    # # print shapes
    # @show size(A)
    # @show size(B)

    Fx = d2Tx - eval_coeffs(traj_s.spl_x.basis, traj_s.spl_x.coefs, ts, BK.Derivative(2))
    Fy = d2Ty - eval_coeffs(traj_s.spl_y.basis, traj_s.spl_y.coefs, ts, BK.Derivative(2))

    H = 2.0 .* Bx' * Pxx * Bx + By' * Pxy * Bx + By' * Pyy * By
    # g = Fx' * Pxx * Bx + Fx' * Pxy * By + Fy' * Pxy * Bx + Fy' * Pyy * By

    # convert this python to julia
    # (Fx.T @ Pxx @ Bx + Fy.T @ Pxy @ By + Fy.T @ Pyy @ By) + (Bx.T @ Pxx @ Fx + By.T @ Pxy @ Fx + By.T @ Pyy @ Fy).T
    g = (Fx' * Pxx * Bx + Fy' * Pxy * By + Fy' * Pyy * By) + (Bx' * Pxx * Fx + By' * Pxy * Fx + By' * Pyy * Fy)'
    # convert matrix to vector
    g = g[1, :]
    # @show mean(g)
    return H, g
end

min_curvature_cost (generic function with 1 method)

We encode the track boundary constraints in frenet frame. For each point $p(x,y)$ and its original position $p_0(x,y)$ and heading $\theta$,

$$
\begin{align}
& s_{\text{right}} \le R(p-p_0) \le s_{\text{left}} \\
\text{where} \quad & R = x\sin\theta+y\cos\theta
\end{align}
$$

R finds the lateral coordinate of the point in frenet frame, with left being positive.

The equation is rearranged to form a standard QP constraint

$$
\begin{align}
& a_{\text{lb}} \le AZ \le a_{\text{ub}}
\end{align}
$$

In [8]:
function track_boundary_constraint(Z::AbstractVector{T}, traj_s::SplineTrajectory, traj_d::DiscreteTrajectory, ts::Vector{Float64}) where {T}
    M = size(traj_d.data, 1)
    N = length(Z) ÷ 2
    # left_x = traj_d.data[:, TRAJ_LEFT_BOUND_X]
    # left_y = traj_d.data[:, TRAJ_LEFT_BOUND_Y]
    # right_x = traj_d.data[:, TRAJ_RIGHT_BOUND_X]
    # right_y = traj_d.data[:, TRAJ_RIGHT_BOUND_Y]
    # min_bound = zeros(eltype(traj_d.data), 2 * M)
    # min_bound[begin:M] = min.(left_x, right_x)
    # min_bound[M+1:end] = min.(left_y, right_y)
    # max_bound = zeros(eltype(traj_d.data), 2 * M)
    # max_bound[begin:M] = max.(left_x, right_x)
    # max_bound[M+1:end] = max.(left_y, right_y)
    
    A2 = zeros(eltype(traj_d.data), 2 * M, 2 * N)
    B = FD.jacobian(coef->eval_coeffs(traj_s.spl_x.basis, coef, ts), traj_s.spl_x.coefs)
    A2[begin:M, begin:N] = B
    A2[M+1:end, N+1:end] = B

    cosθ = cos.(traj_d.data[:, TRAJ_YAW])
    sinθ = sin.(traj_d.data[:, TRAJ_YAW])
    # A11 = hcat(diagm(cosθ), diagm(sinθ) .* -1.0)
    # A12 = hcat(diagm(sinθ), diagm(cosθ))
    A11 = hcat(diagm(cosθ), diagm(sinθ))
    A12 = hcat(diagm(sinθ) .* -1.0, diagm(cosθ))
    A1 = vcat(A11, A12)
    A = A1 * A2
    b = -1.0 * A * Z

    Δr = traj_d.data[:, TRAJ_RIGHT_BOUND_X:TRAJ_RIGHT_BOUND_Y] - traj_d.data[:, TRAJ_X:TRAJ_Y]
    Δl = traj_d.data[:, TRAJ_LEFT_BOUND_X:TRAJ_LEFT_BOUND_Y] - traj_d.data[:, TRAJ_X:TRAJ_Y]
    min_bound = traj_d.data[:, TRAJ_LAT_ACC] .* -1.0 .* norm.(eachrow(Δr)) .+ 2.0
    min_bound = vcat(min_bound, ones(eltype(min_bound), M) .* -1.0) - b
    max_bound = traj_d.data[:, TRAJ_LON_ACC] .* norm.(eachrow(Δl)) .- 2.0
    max_bound = vcat(max_bound, ones(eltype(max_bound), M) .* 1.0) - b

    return A, min_bound, max_bound, B
end

track_boundary_constraint (generic function with 1 method)

In [10]:
@testset "Minimum Curvature QP" begin
    interval = 5.0 # discretization interval
    
    x, y, z = load_raw_trajectory("examples/race_track/monza/MONZA_UNOPTIMIZED_LINE_enu.csv")
    traj_s = SplineTrajectory(x, y, 3)

    x, y, z = load_raw_trajectory("examples/race_track/monza/MONZA_LEFT_BOUNDARY_enu.csv")
    left_s = SplineTrajectory(x, y, 3)

    x, y, z = load_raw_trajectory("examples/race_track/monza/MONZA_RIGHT_BOUNDARY_enu.csv")
    right_s = SplineTrajectory(x, y, 3)

    # x, y, z = load_raw_trajectory("examples/race_track/putnam/PUTNAM_A_CENTER_enu.csv")
    # traj_s = SplineTrajectory(x, y, 3)

    # x, y, z = load_raw_trajectory("examples/race_track/putnam/PUTNAM_BOUND_A_LEFT_enu.csv")
    # left_s = SplineTrajectory(x, y, 3)

    # x, y, z = load_raw_trajectory("examples/race_track/putnam/PUTNAM_BOUND_A_RIGHT_enu.csv")
    # right_s = SplineTrajectory(x, y, 3)
    
    traj_d, ts= discretize_trajectory(traj_s, interval, false)
    left_d, _ = discretize_trajectory(left_s, 0.5)
    right_d, _ = discretize_trajectory(right_s, 0.5)

    set_trajectory_bounds(traj_d, left_d, right_d)

    M = size(traj_d.data, 1)

    coeffs_x = traj_s.spl_x.coefs
    coeffs_y = traj_s.spl_y.coefs

    # keep a copy of the original coefficients
    coeffs_x_orig = copy(coeffs_x)
    coeffs_y_orig = copy(coeffs_y)

    N = length(coeffs_x)
    Z = zeros(eltype(coeffs_x), N * 2)
    Z[begin:N] .= coeffs_x
    Z[N+1:end] .= coeffs_y

    A, min_bound, max_bound, B = track_boundary_constraint(Z, traj_s, traj_d, ts)
    q = zeros(eltype(coeffs_x), N * 2)

    A2 = zeros(eltype(traj_d.data), 2 * M, 2 * N)
    A2[begin:M, begin:N] = B
    A2[M+1:end, N+1:end] = B

    Z_optm = Z

    for i=1:4
        println("Iteration $i")
        # obtain and normalize H to norm of 1
        H, g = min_curvature_cost(Z_optm, traj_s, ts)
        # check if H is positive semi-definite
        # @test eigvals(H) >= 0.0
        H = H ./ norm(H)
        g = g ./ norm(H)
        m = OSQP.Model()
        # @show H
        # @show size(g)
        # time to solve the QP
        start = time()
        OSQP.setup!(m; P=sparse(H), q=q, A=sparse(A), l=min_bound, u=max_bound, verbose=false)
        OSQP.warm_start!(m; x=Z)
        results = OSQP.solve!(m)
        println("Solved in $(time() - start) seconds")
        # @show norm(results.x .- Z_optm)
        Z_optm = results.x
        traj_s.spl_x.coefs .= Z_optm[begin:N]
        traj_s.spl_y.coefs .= Z_optm[N+1:end]

        # if i == 2
        #     break
        # end

        # prepare the next iteration
        traj_d, ts = discretize_trajectory(traj_s, interval, false)
        set_trajectory_bounds(traj_d, left_d, right_d)
        M = size(traj_d.data, 1)
        coeffs_x = traj_s.spl_x.coefs
        coeffs_y = traj_s.spl_y.coefs
        N = length(coeffs_x)
        Z = zeros(eltype(coeffs_x), N * 2)
        Z[begin:N] .= coeffs_x
        Z[N+1:end] .= coeffs_y
        A, min_bound, max_bound, B = track_boundary_constraint(Z, traj_s, traj_d, ts)
        q = zeros(eltype(coeffs_x), N * 2)
        A2 = zeros(eltype(traj_d.data), 2 * M, 2 * N)
        A2[begin:M, begin:N] = B
        A2[M+1:end, N+1:end] = B
        # Z_optm = Z
    end

    plt.show()
    traj_optm = A2 * Z_optm
    plt.plot(traj_optm[1:M], traj_optm[M+1:end], ".")
    plt.plot(traj_d.data[:, TRAJ_LEFT_BOUND_X], traj_d.data[:, TRAJ_LEFT_BOUND_Y])
    plt.plot(traj_d.data[:, TRAJ_RIGHT_BOUND_X], traj_d.data[:, TRAJ_RIGHT_BOUND_Y])
    # plt.plot(traj_d.data[:, TRAJ_X], traj_d.data[:, TRAJ_Y])
    plt.gca().set_aspect("equal")
    plt.show()

    @test min_bound <= A * Z <= max_bound
    @test min_bound <= A * Z_optm <= max_bound

    # save the original coefficients to experiment_data/original_coeffs.csv
    coeffs_orig = [coeffs_x_orig.data coeffs_y_orig.data]
    writedlm("experiment_data/original_coeffs_monza.csv", coeffs_orig, ',')
    # save the optimized coefficients to experiment_data/optimized_coeffs.csv
    coeffs_optm = [traj_s.spl_x.coefs.data traj_s.spl_y.coefs.data]
    writedlm("experiment_data/optimized_coeffs_monza.csv", coeffs_optm, ',')
    # save the optimized discrete trajectory to experiment_data/optimized_trajectory.csv
    traj_optm = [traj_optm[1:M] traj_optm[M+1:end]]
    writedlm("experiment_data/optimized_trajectory_monza.csv", traj_optm, ',')
end

Iteration 1
Solved in 0.003735065460205078 seconds


Iteration 2
Solved in 0.003873109817504883 seconds


Iteration 3
Solved in 0.0045719146728515625 seconds


Iteration 4
Solved in 0.004517078399658203 seconds
Discretizing trajectory: 20.0 %


Discretizing trajectory: 40.0 %


Discretizing trajectory: 60.0 %


Discretizing trajectory: 80.0 %


Discretizing trajectory: 100.0 %


Test Summary:        | Pass  Total   Time
Minimum Curvature QP |    2      2  55.0s


Test.DefaultTestSet("Minimum Curvature QP", Any[], 2, false, false, true, 1.694917530336437e9, 1.694917585287215e9, false)